<a href="https://colab.research.google.com/github/anil-avvaru-cool/code-gen-ai/blob/main/code_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install transformers

In [1]:
from huggingface_hub import login
import os
from IPython.display import Markdown, display
from google.colab import userdata

local_debug = True
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

def connect_to_huggingface():
    hf_token_val = os.getenv('HF_TOKEN')
    login(token=hf_token_val)
    if local_debug :
        print(f"hf_token_val len", len(hf_token_val))
connect_to_huggingface()

def printmd(string):
    display(Markdown(string))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


hf_token_val len 37


In [2]:
prompt = """
You are an expert in LotusScript and HCL Domino.

Explain the purpose and behavior of the following LotusScript agent.
List:
- What the code does
- Important side effects
- External dependencies
- Potential risks or failure points

<<CODE>>
Option Public
Use "IssueUtils"
Use "NotificationUtils"
Sub Initialize
    On Error GoTo ErrHandler
    Dim session As New NotesSession
    Dim db As NotesDatabase
    Set db = session.CurrentDatabase

    Dim doc As NotesDocument
    Set doc = db.CreateDocument()
    doc.Form = "IssueForm"
    doc.IssueID = GenerateIssueID(session)
    doc.Title = "Sample issue created by CreateIssueAgent"
    Dim rtitem As NotesRichTextItem
    Set rtitem = New NotesRichTextItem(doc, "Description")
    Call rtitem.AppendText("This issue was created by CreateIssueAgent for testing.")
    doc.Priority = "Medium"
    doc.Status = "New"
    doc.CreatedBy = session.CommonUserName
    doc.CreatedDate = Now
    doc.AssignedTo = session.CommonUserName
    doc.NotifyOnCreate = "1"
    Call doc.Save(True, False)

    If (doc.NotifyOnCreate(0) = "1") Then
        Call SendIssueNotification(db, doc.AssignedTo(0), "New Issue Assigned: " & doc.IssueID(0), "Title: " & doc.Title(0))
    End If

    Messagebox "Issue created: " & doc.IssueID(0), , "CreateIssueAgent"
    Exit Sub
ErrHandler:
    Messagebox "Error in CreateIssueAgent: " & Err & " - " & Error$, , "Agent Error"
End Sub
<<CODE>>
"""

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "codellama/CodeLlama-7b-Instruct-hf"   # example handle — use the exact HF repo you have access to
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map="auto")

nlp = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

formattedPrompt = [
    {"role": "user", "content": prompt},
]

outout = nlp(formattedPrompt, do_sample=False, max_new_tokens=300)
print(outout[0]['generated_text'])
print(outout[0]["generated_text"][0])
printmd(outout[0]["generated_text"][1]["content"])

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'role': 'user', 'content': '\nYou are an expert in LotusScript and HCL Domino.\n\nExplain the purpose and behavior of the following LotusScript agent.\nList:\n- What the code does\n- Important side effects\n- External dependencies\n- Potential risks or failure points\n\n<<CODE>>\nOption Public\nUse "IssueUtils"\nUse "NotificationUtils"\nSub Initialize\n    On Error GoTo ErrHandler\n    Dim session As New NotesSession\n    Dim db As NotesDatabase\n    Set db = session.CurrentDatabase\n\n    Dim doc As NotesDocument\n    Set doc = db.CreateDocument()\n    doc.Form = "IssueForm"\n    doc.IssueID = GenerateIssueID(session)\n    doc.Title = "Sample issue created by CreateIssueAgent"\n    Dim rtitem As NotesRichTextItem\n    Set rtitem = New NotesRichTextItem(doc, "Description")\n    Call rtitem.AppendText("This issue was created by CreateIssueAgent for testing.")\n    doc.Priority = "Medium"\n    doc.Status = "New"\n    doc.CreatedBy = session.CommonUserName\n    doc.CreatedDate = Now\n  

NameError: name 'outout' is not defined